In [7]:
import gzip, json
with gzip.open('/data/NPMvulnerabilities/NPMPackages.json.gz', 'rb') as f:
    fc = f.read().decode("utf-8")
from json import JSONDecoder, JSONDecodeError
import re

NOT_WHITESPACE = re.compile(r'[^\s]')

def decode_stacked(document, pos=0, decoder=JSONDecoder()):
    while True:
        match = NOT_WHITESPACE.search(document, pos)
        if not match:
            return
        pos = match.start()

        try:
            obj, pos = decoder.raw_decode(document, pos)
        except JSONDecodeError:
            # do something sensible if there's some error
            print("DECODE_STACKED() FAILED")
            pass
        yield obj

In [9]:
for obj in decode_stacked(fc):
    ind = 0
    try: versions = list(obj['versions'].keys())
    except: continue
    try: release_dates = obj['time']
    except: continue
    info = {key: {'release_date': release_dates[key]} for key in release_dates if key not in ['created', 'modified']}
    for key in versions:
        try:
            info[key]['dependencies'] = list(obj['versions'][key]['dependencies'].keys())
            if pkg in info[key]['dependencies']: ind = 1
        except: continue
    if ind :
        dependent[pkg].append({obj['name']: info})

In [ ]:
for k in dependent.keys():
    if k == 'ajv-keywords':
        print(k, dependent[k][0])
        break

In [ ]:
#user-info
#    pji
#        version 1.0
#        date
#        
#        version 1.1
#        date
#    gifnoc
#        version
#        date
        

In [26]:
#pkgs = ['htmlnano', 'object.map', 'babylon-walk', 'wix-style-react', 'threads', 'minimist-options', 'lead', '@babel/polyfill', '@babel/helper-define-map', 'cloudscraper']
pkgs = ['ajv-keywords']   
dependent = {}
for pkg in pkgs:
    dependent[pkg] = []
    for obj in decode_stacked(fc):
        ind = 0
        try: versions = list(obj['versions'].keys())
        except: continue
        try: release_dates = obj['time']
        except: continue
        info = {key: {'release_date': release_dates[key]} for key in release_dates if key not in ['created', 'modified']}
        for key in versions:
            try:
                info[key]['dependencies'] = list(obj['versions'][key]['dependencies'].keys())
                if pkg in info[key]['dependencies']: ind = 1
            except: continue
        if ind :
            dependent[pkg].append({obj['name']})

In [27]:
for k in dependent.keys():
    if k == 'ajv-keywords':
        print(k, dependent[k][:20])
        print("len = ", len(dependent[k]))
        break

ajv-keywords [{'jsonscript-proxy'}, {'senasaikou-yeoman'}, {'webpack-egoist'}, {'alex-d.js'}, {'advanced-image-loader'}, {'webpack-addons-ylvis'}, {'primeng-custom'}, {'jicli'}, {'@lgeiger/electron-builder'}, {'@touched/map-tool-project'}, {'generator-arc'}, {'node-private-tools'}, {'fhir2'}, {'webpack-ie8'}, {'mocoolka-tools'}, {'misstep'}, {'@cysonius/schema-utils'}, {'ng2-dnd-kokatsuna'}, {'qweb-client'}, {'@johnf/electron-builder'}]
len =  157
